In [52]:
import plotly.io as pio
pio.renderers.default = "notebook"

In [ ]:
import pandas as pd

# Load directly from GitHub
url = "https://raw.githubusercontent.com/frederickottto/Project2DW/main/atp_leaderboards_until_2024.csv"
df = pd.read_csv(url)

# Preview data
df.head()


In [ ]:
df

In [ ]:
# Check column names and types
df.info()

# Check basic stats
df.describe(include='all')

In [ ]:
# Count missing values
print("Missing values per column:")
print(df.isnull().sum())

# Count duplicates
print(f"Duplicate rows: {df.duplicated().sum()}")


In [ ]:
# Split into Serve and Return datasets
serve_df = df[df['Board_Type'] == 'serve'].reset_index(drop=True)
return_df = df[df['Board_Type'] == 'return'].reset_index(drop=True)

# Save to separate CSV files
serve_df.to_csv("serve_leaderboard.csv", index=False)
return_df.to_csv("return_leaderboard.csv", index=False)

print("Files saved: 'serve_leaderboard.csv' and 'return_leaderboard.csv'")


In [ ]:
url = "https://github.com/frederickottto/Project2DW/raw/refs/heads/main/serve_leaderboard.csv"
serve_df = pd.read_csv(url)
serve_df.head()

In [ ]:
serve_df

In [ ]:
serve_df.isnull().sum()

In [ ]:
url = "https://github.com/frederickottto/Project2DW/raw/refs/heads/main/return_leaderboard.csv"
return_df = pd.read_csv(url)
return_df.head()

In [ ]:
# List of return-related columns to drop
return_columns = [
    'Return Rating',
    '% 1st Serve Return Points Won',
    '% 2nd Serve Return Points Won',
    '% Return Games Won',
    '% Break Points Converted',
    'Board_Type'
]

# Drop them from serve_df
serve_df = serve_df.drop(columns=return_columns)

# Check result
serve_df.head()

In [ ]:
serve_columns = [
    'Serve Rating', '% 1st Serve', '% 1st Serve', '% 1st Serve Points Won','% 2st Serve Points Won', '% Service Games Won', 'Avg. Aces/ Match', 'Avg. Double Faults/Match'
]

return_df = return_df.drop(columns=serve_columns)

return_df.head()

In [64]:
return_df = return_df.drop(columns='Board_Type')

In [ ]:
return_df.head()

In [ ]:
serve_df.head()

In [ ]:
# Reorder columns: move 'Year' to come right after 'Player'
cols = serve_df.columns.tolist()
cols.remove('Year')
new_order = cols[:1] + ['Year'] + cols[1:]
serve_df = serve_df[new_order]

# Check result
serve_df.head()


In [68]:
def validate_leaderboard(df, required_columns, percent_columns, numeric_columns, name=""):
    import pandas as pd

    print(f"\n Validating {name} leaderboard...")

    # 1. Check required columns exist
    for col in required_columns:
        if col not in df.columns:
            print(f" Missing column: {col}")
    
    # 2. Check for missing values
    print("\n Missing values per column:")
    print(df[required_columns].isnull().sum())

    # 3. Drop duplicates
    before = len(df)
    df = df.drop_duplicates()
    print(f"\n Removed {before - len(df)} duplicate rows.")

    # 4. Convert percentage strings to floats
    for col in percent_columns:
        if df[col].dtype == object:
            df[col] = (
                df[col].str.replace('%', '', regex=False)
                      .astype(float)
            )

    # 5. Convert numeric columns
    for col in numeric_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # 6. Ensure Year is integer
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')

    # 7. Custom checks: percent values in 0–100, ratings non-negative
    print("\n Out-of-bound checks:")
    for col in percent_columns:
        out_of_bounds = df[(df[col] < 0) | (df[col] > 100)]
        if not out_of_bounds.empty:
            print(f" - {col}: {len(out_of_bounds)} rows outside 0–100 range.")
    
    for col in numeric_columns:
        if 'Rating' in col or 'Aces' in col or 'Faults' in col:
            neg = df[df[col] < 0]
            if not neg.empty:
                print(f" - {col}: {len(neg)} rows have negative values.")

    print(f"\n Validation complete for {name} leaderboard.")
    return df


In [ ]:
serve_required = ['Player', 'Year', 'Serve Rating', '% 1st Serve', '% 1st Serve Points Won',
                  '% 2st Serve Points Won', '% Service Games Won',
                  'Avg. Aces/ Match', 'Avg. Double Faults/Match']

serve_percent = ['% 1st Serve', '% 1st Serve Points Won', '% 2st Serve Points Won', '% Service Games Won']
serve_numeric = ['Serve Rating', 'Avg. Aces/ Match', 'Avg. Double Faults/Match']

serve_df = validate_leaderboard(serve_df, serve_required, serve_percent, serve_numeric, name="Serve")


In [ ]:
return_required = ['Player', 'Year', 'Return Rating', '% 1st Serve Return Points Won',
                   '% 2nd Serve Return Points Won', '% Return Games Won', '% Break Points Converted']

return_percent = ['% 1st Serve Return Points Won', '% 2nd Serve Return Points Won',
                  '% Return Games Won', '% Break Points Converted']
return_numeric = ['Return Rating']

return_df = validate_leaderboard(return_df, return_required, return_percent, return_numeric, name="Return")


In [ ]:
serve_df


In [ ]:
serve_df = serve_df.rename(columns={'Unnamed: 0': 'Standing'})
cols = serve_df.columns.tolist()
cols.remove('Standing')
cols.insert(cols.index('Player') + 1, 'Standing')
serve_df = serve_df[cols]
serve_df

In [ ]:
return_df

In [ ]:
# Add Standing based on row order per year
return_df["Standing"] = (
    return_df.groupby("Year")
    .cumcount() + 1
)

# Move 'Standing' after 'Player'
cols = return_df.columns.tolist()
cols.remove("Standing")
cols.insert(cols.index("Player") + 1, "Standing")
return_df = return_df[cols]

# Preview result
return_df


In [75]:
#serve_df.to_csv("serve_leaderboard_cleaned.csv", index=False)
#return_df.to_csv("return_leaderboard_cleaned.csv", index=False)


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np


# Filter for 2024
serve_2024 = serve_df[serve_df["Year"] == 2024].sort_values("Standing")
return_2024 = return_df[return_df["Year"] == 2024].sort_values("Standing")

# 1. Serve Line Plot
fig1 = px.line(
    serve_2024,
    x="Standing",
    y="% 1st Serve",
    title="% 1st Serve by Standing (2024)",
    markers=True
)
fig1.show()

# 2. Serve Bar Chart (Top 10)
fig2 = px.bar(
    serve_2024.head(10),
    x="Player",
    y="% Service Games Won",
    title="% Service Games Won – Top 10 Servers (2024)",
    color="Player"
)
fig2.show()

# 3. Serve Scatter Plot + Best Fit
x = serve_2024["% 1st Serve"]
y = serve_2024["% 2st Serve Points Won"]
coeffs = np.polyfit(x, y, 1)
trend = np.poly1d(coeffs)
fig3 = px.scatter(
    serve_2024,
    x="% 1st Serve",
    y="% 2st Serve Points Won",
    hover_name="Player",
    title="% 2nd Serve Points Won vs % 1st Serve (2024)",
    color="Player"
)
fig3.add_trace(go.Scatter(
    x=x,
    y=trend(x),
    mode="lines",
    name="Best Fit Line",
    line=dict(color="black", dash="dash")
))
fig3.show()

# 4. Return Line Plot
fig4 = px.line(
    return_2024,
    x="Standing",
    y="% Return Games Won",
    title="% Return Games Won by Standing (2024)",
    markers=True
)
fig4.show()

# 5. Return Bar Chart (Top 10)
fig5 = px.bar(
    return_2024.head(10),
    x="Player",
    y="% Break Points Converted",
    title="% Break Points Converted – Top 10 Returners (2024)",
    color="Player"
)
fig5.show()

# 6. Return Scatter Plot + Best Fit
x_r = return_2024["% 1st Serve Return Points Won"]
y_r = return_2024["% 2nd Serve Return Points Won"]
coeffs_r = np.polyfit(x_r, y_r, 1)
trend_r = np.poly1d(coeffs_r)
fig6 = px.scatter(
    return_2024,
    x="% 1st Serve Return Points Won",
    y="% 2nd Serve Return Points Won",
    hover_name="Player",
    title="% 2nd Serve Return Points Won vs % 1st Serve Return (2024)",
    color="Player"
)
fig6.add_trace(go.Scatter(
    x=x_r,
    y=trend_r(x_r),
    mode="lines",
    name="Best Fit Line",
    line=dict(color="black", dash="dash")
))
fig6.show()


